# Noise robustness of BioPCA and IBCM to transient background odors
Consider a simple three-odor case with turbulent statistics, add and remove new background odors on a slow time scale. 

This requires defining a new background function where, on top of the regular background odors, we add an extra, slow, turbulent odor which, every time it reappears, has a new orientation. Make its time scale much slower and its concentration much smaller. 
Make it a unit norm vector along one of the $n_R$ ORNs. Hence, it can be chosen with one uniform random sample, to choose the ORN identity. In total, we need two extra pairs of random samples per step, at most: one pair to update t, c of that transient odor, and one pair for ORN selection. 

(Next step of complexity: generate a new odor each time. Would need to pass a random generator as a parameter? OK for this notebook but would fail for any future multiprocessing. Anyways, first try the simpler step here where the new odor = one ORN is activated. )

## Imports

In [ ]:
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from time import perf_counter
import os, json
from math import floor as floor_int

from modelfcts.ibcm import (
    integrate_inhib_ibcm_network_options,
    ibcm_respond_new_odors,
    compute_mbars_cgammas_cbargammas,
    ibcm_respond_new_odors
)
from modelfcts.ibcm_analytics import (
    fixedpoint_thirdmoment_exact, 
    fixedpoint_thirdmoment_perturbtheory,
    ibcm_fixedpoint_w_thirdmoment, 
    ibcm_all_largest_eigenvalues
)
from modelfcts.biopca import (
    integrate_inhib_ifpsp_network_skip,
    build_lambda_matrix,
    biopca_respond_new_odors
)
from modelfcts.average_sub import (
    integrate_inhib_average_sub_skip, 
    average_sub_respond_new_odors
)
from modelfcts.ideal import (
    find_projector, 
    find_parallel_component, 
    ideal_linear_inhibitor, 
    compute_ideal_factor
)
from modelfcts.checktools import (
    check_conc_samples_powerlaw_exp1,
    compute_pca_meankept, 
    compute_projector_series, 
    analyze_pca_learning
)
from modelfcts.backgrounds import (
    update_powerlaw_times_concs, 
    logof10, 
    sample_background_powerlaw,
    sample_ss_conc_powerlaw, 
    decompose_nonorthogonal_basis, 
    update_alternating_inputs, 
    generate_odorant, 
    update_tc_odor
)
from modelfcts.tagging import (
    project_neural_tag, 
    create_sparse_proj_mat, 
    SparseNDArray, 
    tags_list_to_csr_matrix
)
from utils.statistics import seed_from_gen
from modelfcts.distribs import (
    truncexp1_inverse_transform, 
    truncexp1_density, 
    truncexp1_average,
    powerlaw_cutoff_inverse_transform
)
from utils.smoothing_function import (
    moving_average, 
    moving_var
)
from simulfcts.plotting import (
    plot_cbars_gammas_sums, 
    plot_cbars_gamma_series, 
    plot_3d_series, 
    plot_w_matrix, 
    plot_background_norm_inhibition, 
    plot_background_neurons_inhibition, 
    plot_pca_results, 
    hist_outline
)
from simulfcts.analysis import compute_back_reduction_stats
from utils.metrics import jaccard, l2_norm

In [ ]:
def update_transient_odor(tc_iu, dt, unif, n_orn, *args):
    """ Update the transient odor. It has a turbulent process for its
    whiff/blank time t and concentration c, and a random uniform
    uncorrelated sampling of the identity i of the active ORN. 
    
    Args:
        tc_iu (np.ndarray): 2x2 array. First row is (t, c) of the
            transient odor, second row is the active ORN index 
            and a dummy number. 
        dt (float): time step
        unif (np.ndarray): 2x2 array of U(0, 1) samples
            for t, c update and ORN selection.
        n_orn (int): number of ORNs to select from. 
        parameters: t_w_lo, t_w_hi, t_b_lo, t_b_hi, c0, alpha
    """
    twlo, twhi, tblo, tbhi, c0, alpha = args
    # Update needed if dt or less left to the wait time.
    if (tc_iu[0, 0] - dt) <= 0:
        # Determine whether we were in a whiff or a blank
        if tc_iu[0, 1] > 0 :  # we were in a whiff
            # Pull t_b, duration of new blank
            tc_iu[0, 0:1] = powerlaw_cutoff_inverse_transform(unif[0, 0:1], tblo, tbhi)
            # Set c to zero
            tc_iu[0, 1] = 0.0
        else:  # we were in a blank
            # Pull t_w, duration of new whiff
            tc_iu[0, 0:1] = powerlaw_cutoff_inverse_transform(unif[0, 0:1], twlo, twhi)
            # Set conc c of the whiff
            tc_iu[0, 1:2] = truncexp1_inverse_transform(unif[0, 1:], c0, alpha)
            # Change active ORN
            tc_iu[1, 0] = floor_int(unif[1, 0] * n_orn) + 1e-12  # ensure conversion to float and back is OK
            tc_iu[1, 1] = unif[1, 1]
    else:
        tc_iu[0, 0] = tc_iu[0, 0] - dt
    return tc_iu

def update_powerlaw_one_orn_noise(tc_bk, params_bk, noises, dt):
    """
    Simulate turbulent odors by pulling wait times until the end of a whiff
        or until the next blank, and a concentration of the whiff.
        For each odor, check whether the time left until switch is <= zero;
        if so, pull either
            - another wait time t_w if current c=0, and pull the new c > 0
              (we were in a blank and are starting a whiff)
            - another wait time t_b if current c > 0, and set c = 0
              (we were in a whiff and are starting a blank)
        Otherwise, decrement t by dt and don't change c.
    Then simulate a turbulent concentration of an extra transient odor vector, 
    which will be activation of a single ORN, selected by the last pair of
    uniform random samples. 

    Args:
        tc_bk (np.ndarray): array of t, c for each odor in the background,
            where t = time left until next change, c = current concentration
            of the odor. 
            Next row is a (t, c) pair for the transient odor
            Last row is the identity of the active ORN and a dummy number. 
            Shaped [n_odors + 2, 2]
        params_bk (list): contains the following elements (a lot needed!):
            whiff_tmins (np.ndarray): lower cutoff in the power law
                of whiff durations, for each odor
            whiff_tmaxs (np.ndarray): upper cutoff in the power law
                of whiff durations, for each odor
            blank_tmins (np.ndarray): same as whiff_tmins but for blanks
            blank_tmaxs (np.ndarray): same as whiff_tmaxs but for blanks
            c0s (np.ndarray): c0 concentration scale for each odor
            alphas (np.ndarray): alpha*c0 is the lower cutoff of p_c
            noise_ampli (1-element array): amplitude (standard dev.) 
                of Gaussian white noise added to each ORN on top of the background. 
            vecs (np.ndarray): 2d array where each row is one of the
                possible input vectors
        noises (np.ndarray): fresh U(0, 1) samples, shaped [n_odors + [n_R//2], 2],
            in case we need to pull a new t and/or c.
            TODO: most noises are wasted; for now memory isn't an issue
            but this is a place where the code can be optimized a lot.
        dt (float): time step duration, in simulation units
    """
    # Update one odor's t and c at a time, if necessary
    tc_bk_new = np.zeros(tc_bk.shape)
    # infer number of odors based on number of dimensions
    vecs_nu = params_bk[-1]
    n_odors, n_orn = vecs_nu.shape
    for i in range(n_odors):
        tc_bk_new[i] = update_tc_odor(tc_bk[i], dt, noises[i, :2],
                                *[p[i] for p in params_bk[:6]])
    # Update the transient odor concentration too:
    tc_bk_new[n_odors:] = update_transient_odor(tc_bk[n_odors:], dt, 
                                noises[n_odors:, :2], n_orn, 
                                *[p[n_odors] for p in params_bk[:6]])
    # Compute backgound vector (even if it didn't change)
    # TODO: this could be optimized too by giving the current back vec
    # as an input, but this requires editing the ibcm simulation functions
    new_bk_vec = np.dot(tc_bk_new[:n_odors, 1], vecs_nu)
    # Add transient ORN activation
    orn_index = int(tc_bk_new[n_odors+1, 0])
    new_bk_vec[orn_index] += tc_bk_new[n_odors, 1]  # concentration
    return new_bk_vec, tc_bk_new

### Aesthetic parameters

In [ ]:
#plt.style.use(['dark_background'])
plt.rcParams["figure.figsize"] = (4.5, 3.0)

In [ ]:
models = ["ibcm", "biopca", "avgsub", "ideal", "orthogonal", "none"]
model_nice_names = {
    "ibcm": "IBCM",
    "biopca": "BioPCA",
    "avgsub": "Average",
    "ideal": "Ideal",
    "orthogonal": "Orthogonal",
    "none": "None"
}
model_colors = {
    "ibcm": "xkcd:turquoise",
    "biopca": "xkcd:orangey brown",
    "avgsub": "xkcd:navy blue",
    "ideal": "xkcd:powder blue",
    "orthogonal": "xkcd:pale rose",
    "none": "grey"
}

### Initialization

In [ ]:
# Initialize common simulation parameters
n_dimensions = 25  # Half the real number for faster simulations
n_components = 6  # Number of background odors

inhib_rates = [0.0001, 0.00002]  # alpha, beta  [0.00025, 0.00005]

# Simulation duration
duration = 360000.0
deltat = 1.0
n_chunks = 1
skp = 10 * int(1.0 / deltat)

# Common model options
activ_function = "identity"  #"ReLU"

# Background process
update_fct = update_powerlaw_one_orn_noise

# Choose randomly generated background vectors
rgen_meta = np.random.default_rng(seed=0x8896ce0154295ba29df7e93dc277af2e)
#rgen_meta = np.random.default_rng(seed=0x85dfce01542492a29df7e93dc277ad2d)
back_components = np.zeros([n_components, n_dimensions])
for i in range(n_components):
    back_components[i] = generate_odorant(n_dimensions, rgen_meta, lambda_in=0.1)
back_components = back_components / l2_norm(back_components).reshape(-1, 1)

# Seed for background simulation, to make sure all models are the same
simul_seed = seed_from_gen(rgen_meta)

# Turbulent background parameters: same rates and constants for all odors
back_params = [
    np.asarray([1.0] * n_components + [1.0]),        # whiff_tmins
    np.asarray([500.] * n_components + [500.0]),       # whiff_tmaxs
    np.asarray([1.0] * n_components + [2.0]),        # blank_tmins
    np.asarray([800.0] * n_components + [800.0]),      # blank_tmaxs
    np.asarray([0.6] * n_components + [0.2]),        # c0s and noise amplitude
    np.asarray([0.5] * (n_components + 1)),        # alphas
]
back_params.append(back_components)

# Initial values of background process variables (t, c for each variable)
init_concs = sample_ss_conc_powerlaw(*back_params[:6], size=1, rgen=rgen_meta)
init_times = powerlaw_cutoff_inverse_transform(
                rgen_meta.random(size=n_components+1), *back_params[2:4])
init_concs[n_components:] = 0.0
tc_init = np.stack([init_times, init_concs.squeeze()], axis=1)
tc_init = np.concatenate([tc_init, np.zeros([1, 2])], axis=0)

# Initial background vector 
init_bkvec = tc_init[:n_components, 1].dot(back_components)
# nus are first in the list of initial background params
init_back_list = [tc_init, init_bkvec]

### Pairwise similarity between background odors
Determines how well-posed the PCA is and how easy it is for the IBCM model to disentangle odors


## IBCM habituation
### IBCM simulation

In [ ]:
# IBCM model parameters
n_i_ibcm = 24  # Number of inhibitory neurons for IBCM case

# Model rates
learnrate_ibcm = 0.00125  #5e-5
tau_avg_ibcm = 1600  # 2000
coupling_eta_ibcm = 0.6/n_i_ibcm
ssat_ibcm = 50.0
k_c2bar_avg = 0.1
decay_relative_ibcm = 0.005
lambd_ibcm = 1.0
ibcm_rates = [
    learnrate_ibcm, 
    tau_avg_ibcm, 
    coupling_eta_ibcm, 
    lambd_ibcm,
    ssat_ibcm, 
    k_c2bar_avg,
    decay_relative_ibcm 
]
ibcm_options = {
    "activ_fct": activ_function, 
    "saturation": "tanh", 
    "variant": "law", 
    "decay": True
}

# Initial synaptic weights: small positive noise
init_synapses_ibcm = 0.3*rgen_meta.standard_normal(size=[n_i_ibcm, n_dimensions])*lambd_ibcm

In [ ]:
def recompute_theta_series(cbser, tau, dt):
    theta = np.zeros([cbser.shape[0], cbser.shape[1]])
    theta[0] = cbser[0]**2
    for i in range(cbser.shape[0]-1):
        theta[i+1] = theta[i] + dt/tau*(cbser[i]*cbser[i] - theta[i])
    return theta

In [ ]:
# Run the IBCM simulations
# Perform successive shorter runs/restarts for memory efficiency
tser_ibcm = []
nuser_ibcm = []
bkvecser_ibcm = []
mser_ibcm = []
cbarser_ibcm = []
wser_ibcm = []
sser_ibcm = []
thetaser_ibcm = []
if n_chunks > 1:
    seed_spawns = np.random.SeedSequence(simul_seed).spawn(10)
else:
    seed_spawns = [simul_seed]
for i in range(n_chunks):
    tstart = perf_counter()
    if i == 0:
        init_vari = init_synapses_ibcm
        init_back = init_back_list
    else:
        init_vari = [mser_ibcm[i-1][-1], thetaser_ibcm[i-1][-1], wser_ibcm[i-1][-1]]
        init_back = [nuser_ibcm[i-1][-1], bkvecser_ibcm[i-1][-1]]
    sim_results = integrate_inhib_ibcm_network_options(
                init_vari, update_fct, init_back, 
                ibcm_rates, inhib_rates, back_params, duration/n_chunks, 
                deltat, seed=seed_spawns[i], noisetype="uniform",  
                skp=skp, **ibcm_options
    )
    tser_ibcm.append(sim_results[0] + i/n_chunks*duration)
    nuser_ibcm.append(sim_results[1])
    bkvecser_ibcm.append(sim_results[2])
    mser_ibcm.append(sim_results[3]) 
    cbarser_ibcm.append(sim_results[4]) 
    thetaser_ibcm.append(sim_results[5])
    wser_ibcm.append(sim_results[6])
    sser_ibcm.append(sim_results[7])
    tend = perf_counter()
    print("Finished chunk", i, "in {:.2f} s".format(tend - tstart))

# Concatenate
tser_ibcm = np.concatenate(tser_ibcm, axis=0)
nuser_ibcm = np.concatenate(nuser_ibcm)
bkvecser_ibcm = np.concatenate(bkvecser_ibcm)
mser_ibcm = np.concatenate(mser_ibcm)
cbarser_ibcm = np.concatenate(cbarser_ibcm)
thetaser_ibcm = np.concatenate(thetaser_ibcm)
wser_ibcm = np.concatenate(wser_ibcm)
sser_ibcm = np.concatenate(sser_ibcm)

### IBCM habituation analysis

In [ ]:
# Calculate cgammas_bar and mbars
transient = int(5/6*duration / deltat) // skp
# Dot products \bar{c}_{\gamma} = \bar{\vec{m}} \cdot \vec{x}_{\gamma}
mbarser, c_gammas, cbars_gamma = compute_mbars_cgammas_cbargammas(
                                    mser_ibcm, coupling_eta_ibcm, back_components)
sums_cbars_gamma = np.sum(cbars_gamma, axis=2)
sums_cbars_gamma2 = np.sum(cbars_gamma*cbars_gamma, axis=2)

# Analytical prediction, exact: need moments of nu. Easiest to compute numerically. 
conc_ser = nuser_ibcm[:, :n_components, 1]
# Odors are all iid so we can average over all odors
mean_conc = np.mean(conc_ser)
sigma2_conc = np.var(conc_ser)
thirdmom_conc = np.mean((conc_ser - mean_conc)**3)
moments_conc = [mean_conc, sigma2_conc, thirdmom_conc]

# Analytical prediction
res = fixedpoint_thirdmoment_exact(moments_conc, 1, n_components-1)
c_specif, c_nonspecif = res[:2]
cs_cn = res[:2]

# Count how many dot products are at each possible value. Use cbar = 1.0 as a split. 
split_val = 2.0
cbars_gamma_mean = np.mean(cbars_gamma[transient:], axis=0)
cgammas_bar_counts = {"above": int(np.sum(cbars_gamma_mean.flatten() > split_val)), 
                      "below": int(np.sum(cbars_gamma_mean.flatten() <= split_val))}
print(cgammas_bar_counts)

specif_gammas = np.argmax(np.mean(cbars_gamma[transient:], axis=0), axis=1)
print(specif_gammas)

# Analytical W
analytical_w = ibcm_fixedpoint_w_thirdmoment(inhib_rates, moments_conc, back_components, cs_cn, specif_gammas)

In [ ]:
fig, ax = plt.subplots()
#ax.plot(tser_ibcm[:300], nuser_ibcm[:300, :, 1])
neurons_cmap = sns.color_palette("Greys", n_colors=n_i_ibcm)
for i in range(n_i_ibcm):
    ax.plot(tser_ibcm/1000, thetaser_ibcm[:, i], lw=0.5, color=neurons_cmap[i])
ax.set(xlabel="Time (x1000 steps)", ylabel=r"$\bar{\Theta} = \bar{c}^2$ moving average")
plt.show()
plt.close()

In [ ]:
fig , ax, _ = plot_cbars_gamma_series(tser_ibcm, cbars_gamma, 
                        skp=10, transient=320000 // skp)
# Compare to exact analytical fixed point solution
#ax.set_xlim([350, 360])
ax.axhline(c_specif, ls="--", color="grey", 
           label=r"Analytical $\bar{c}_{\gamma=\mathrm{specific}}$")
ax.axhline(c_nonspecif, ls="--", color="grey", 
           label=r"Analytical $\bar{c}_{\gamma=\mathrm{non}}$")
fig.tight_layout()
leg = ax.legend(loc="upper left", bbox_to_anchor=(1., 1.))

#fig.savefig("figures/powerlaw/cbargammas_series_turbulent_background_example.pdf", 
#            transparent=True, bbox_inches="tight", bbox_extra_artists=(leg,))
plt.show()
plt.close()

In [ ]:
# Correlation between nu's and c's, see if some neurons are specific to odors
# Each neuron turns out to correlate its response to  one concentration
# that means it is specific to that odor. 
cbarser_norm_centered = cbarser_ibcm - np.mean(cbarser_ibcm[transient:], axis=0)
conc_ser_centered = (nuser_ibcm[:, :n_components, 1] 
                     - np.mean(nuser_ibcm[transient:, :n_components, 1], axis=0))
correl_c_nu = np.mean(cbarser_norm_centered[transient:, :, None] 
                      * conc_ser_centered[transient:, None, :], axis=0)

fig, ax = plt.subplots()
img = ax.imshow(correl_c_nu.T)
ax.set(ylabel=r"Component $\gamma$", xlabel=r"Neuron $i$")
fig.colorbar(img, label=r"$\langle (\bar{c}^i - \langle \bar{c}^i \rangle)"
             r"(\nu_{\gamma} - \langle \nu_{\gamma} \rangle) \rangle$", 
            location="top")
fig.tight_layout()
#fig.savefig("figures/powerlaw/specificities_turbulent_background_example.pdf", 
#           transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Check if each component has at least one neuron
for comp in range(n_components):
    print("Number of neurons specific to component {}: {}".format(
            comp, np.sum(np.mean(cbars_gamma[-2000:, :, comp], axis=0) > split_val)))

In [ ]:
fig, axes, _ = plot_background_neurons_inhibition(tser_ibcm[1000:1500], 
                        bkvecser_ibcm[1000:1500], sser_ibcm[1000:1500], skp=1)
plt.show()
plt.close()

In [ ]:
fig, ax, bknorm_ser, snorm_ser = plot_background_norm_inhibition(
                                tser_ibcm, bkvecser_ibcm, sser_ibcm, skp=1)

# Compute noise reduction factor, annotate
transient = 100000 // skp
norm_stats = compute_back_reduction_stats(bknorm_ser, snorm_ser, trans=transient)

print("Mean activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
print("Standard deviation of activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
#fig.savefig("figures/powerlaw/pn_activity_norm_turbulent_background_example.pdf", 
#            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
fig, axes = plot_w_matrix(tser_ibcm, wser_ibcm, skp=100)
fig.tight_layout()
#fig.savefig("figures/powerlaw/w_series_turbulent_background_example.pdf", 
#            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

### Stability of average fixed points
Check the eigenvalues of the jacobian for one neuron, for every possible specificity. There are $2^{n_B}$ possibilities: choosing specific or not for each odor

In [ ]:
all_max_eigenvalues = ibcm_all_largest_eigenvalues(
    moments_conc, ibcm_rates, back_components, m3=1.0, cut=1e-16, options=ibcm_options
)

In [ ]:
fig, ax = plt.subplots()
ibcm_specif_keys = list(all_max_eigenvalues.keys())
ibcm_eig_values = np.asarray([all_max_eigenvalues[a] for a in ibcm_specif_keys])
reals, imags = np.real(ibcm_eig_values), np.imag(ibcm_eig_values)
ibcm_eig_values_specif1 = np.asarray([len(s) == 1 for s in ibcm_specif_keys], dtype=bool)
highlights = ibcm_eig_values_specif1
ax.axvline(0.0, ls="--", color="k", lw=1.0)
ax.axhline(0.0, ls="--", color="k", lw=1.0)
scaleup = 1e3
ax.plot(reals[highlights]*scaleup, imags[highlights]*scaleup, marker="*", mfc="b", mec="b", 
        ls="none", label="One odor", ms=8)
ax.plot(reals[~highlights]*scaleup, imags[~highlights]*scaleup, marker="o", mfc="k", mec="k", 
       ls="none", label="0 or 2+ odors", ms=6)
for side in ("top", "right"):
    ax.spines[side].set_visible(False)
ax.legend(title="Specificity")
ax.set(xlabel=r"$\mathrm{Re}(\lambda_{\mathrm{max}})$    ($\times 10^{-3}$)", 
      ylabel=r"$\mathrm{Im}(\lambda_{\mathrm{max}})$     ($\times 10^{-3}$)")
fig.tight_layout()
plt.show()
plt.close()

## Understand what happens when $W$ blows up

## BioPCA simulation
### BioPCA habituation simulation

In [ ]:
# BioPCA model parameters
n_i_pca = min(n_components * 2, n_dimensions)  # Number of inhibitory neurons for BioPCA case

# Model rates
learnrate_pca = 1e-4  # Learning rate of M
# Choose Lambda diagonal matrix as advised in Minden et al., 2018
# but scale it up to counteract W regularization
lambda_range_pca = 0.5
lambda_max_pca = 11.0
#lambda_max_pca = 1.0
# Learning rate of L, relative to learnrate. Adjusted to Lambda in the integration function
rel_lrate_pca = 2.0  #  / lambda_max_pca**2 
lambda_mat_diag = build_lambda_matrix(lambda_max_pca, lambda_range_pca, n_i_pca)

xavg_rate_pca = learnrate_pca
pca_options = {
    "activ_fct": activ_function, 
    "remove_lambda": False, 
    "remove_mean": True
}
biopca_rates = [learnrate_pca, rel_lrate_pca, lambda_max_pca, lambda_range_pca, xavg_rate_pca]


# Initial synaptic weights: small positive noise
# We selected a seed (out of 40+ tested) giving initial conditions leading to correct PCA
# The model has trouble converging on this background, we're giving as many chances as possible here. 
rgen_pca = np.random.default_rng(seed=0x8b6664612cfeda4a121436fcfbbca449)
init_synapses_pca = rgen_pca.standard_normal(size=[n_i_pca, n_dimensions]) / np.sqrt(n_i_pca)
init_mmat_pca = rgen_pca.standard_normal(size=[n_i_pca, n_dimensions]) / np.sqrt(n_dimensions)
init_lmat_pca = np.eye(n_i_pca, n_i_pca)  # Supposed to be near-identity, start as identity
ml_inits_pca = [init_mmat_pca, init_lmat_pca]

In [ ]:
# Run simulation
sim_results = integrate_inhib_ifpsp_network_skip(
                ml_inits_pca, update_fct, init_back_list, biopca_rates, 
                inhib_rates, back_params, duration, deltat, 
                seed=simul_seed, noisetype="uniform", skp=skp, **pca_options)
(tser_pca, 
 nuser_pca, 
 bkvecser_pca, 
 mser_pca, 
 lser_pca, 
 xser_pca, 
 cbarser_pca, 
 wser_pca, 
 sser_pca) = sim_results

### BioPCA simulation analysis

In [ ]:
# Compare to de-noised PCA series
res = analyze_pca_learning(bkvecser_pca, mser_pca, lser_pca, 
                           lambda_mat_diag, demean=pca_options["remove_mean"])
true_pca, learnt_pca, fser, off_diag_l_avg_abs, align_error_ser = res
denoised_bkvecser_pca = nuser_pca[:, :n_components, 1].dot(back_components)
res2 = analyze_pca_learning(denoised_bkvecser_pca, mser_pca, lser_pca, 
                           lambda_mat_diag, demean=pca_options["remove_mean"])


In [ ]:
np.abs(true_pca[0] - res2[0][0]) / true_pca[0][0]

In [ ]:
from utils.statistics import principal_component_analysis
from modelfcts.checktools import compute_pca_meankept

In [ ]:
fig, axes = plot_pca_results(tser_pca/1000, true_pca, learnt_pca, align_error_ser, off_diag_l_avg_abs)
axes[-1].set_xlabel("Time (x1000 steps)")
axes[0].get_legend().remove()
fig.set_size_inches(fig.get_size_inches()[0], 3*2.5)
plt.show()
plt.close()

In [ ]:
fig, ax, bknorm_ser, snorm_ser = plot_background_norm_inhibition(
                                tser_pca, bkvecser_pca, sser_pca, skp=10)

# Compute noise reduction factor, annotate
transient = 100000 // skp
norm_stats = compute_back_reduction_stats(bknorm_ser, snorm_ser, trans=transient)

print("Mean activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
print("Standard deviation of activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
plt.show()
plt.close()

In [ ]:
fig, axes, _ = plot_background_neurons_inhibition(tser_pca, bkvecser_pca, sser_pca, skp=10)
plt.show()
plt.close()

In [ ]:
fig, axes = plot_w_matrix(tser_pca, wser_pca, skp=10)
plt.show()
plt.close()

### Average background subtraction simulation

In [ ]:
# Average subtraction model parameters
avg_options = {"activ_fct": activ_function}

# Initial synaptic weights: dummy
init_synapses_avg = np.zeros([1, n_dimensions])

In [ ]:
sim_results = integrate_inhib_average_sub_skip(
                init_synapses_avg, update_fct, init_back_list, 
                [], inhib_rates, back_params, duration, deltat,
                seed=simul_seed, noisetype="uniform", skp=skp, **avg_options
)
tser_avg, bkser_avg, bkvecser_avg, wser_avg, sser_avg = sim_results

### Ideal inhibition
The component parallel to the background is reduced to beta / (2*alpha + beta). 

In [ ]:
back_projector = find_projector(back_components.T)
ideal_factor = inhib_rates[1] / (2*inhib_rates[0] + inhib_rates[1])
# Only consider the subspace spanned by odors, not noise
# So need to project on the subspace. 
sser_ideal = bkvecser_ibcm - bkvecser_ibcm.dot(back_projector.T) * (1.0 - ideal_factor)

## Model comparison for background inhibition

## Model comparison for background tagging after habituation
We create a projection matrix, then compute the tag assigned to the background after inhibition by each habituation model, over time. Hopefully, only IBCM inhibits enough to see tags go to zero. 

In [ ]:
# Common parameters
n_kc = 1000
projection_arguments = {
    "kc_sparsity": 0.05,
    "adapt_kc": True,
    "n_pn_per_kc": 3,
    "project_thresh_fact": 0.1
}
proj_mat = create_sparse_proj_mat(n_kc, n_dimensions, rgen_meta)
sser_dict = {
    "ibcm": sser_ibcm, 
    "biopca": sser_pca, 
    "avgsub": sser_avg, 
    "none": bkvecser_ibcm, 
    "ideal": sser_ideal
}

## Model comparison for new odor recognition

In [ ]:
def find_snap_index(dt, skip, times):
    """ Find nearest multiple of dt*skip to each time in times """
    return np.around(times / (dt*skip)).astype(int)

In [ ]:
# Generate new odors, select test times, etc.
# New odors tested
n_new = 100
new_odors = generate_odorant([n_new, n_dimensions], rgen_meta, lambda_in=0.1)
new_odors /= l2_norm(new_odors)[:, None]

# Test times
n_test_times = 10
start_test_t = duration - n_test_times * 2000.0
test_times = np.linspace(start_test_t, duration, n_test_times)
test_times -= deltat*skp
test_idx = find_snap_index(deltat, skp, test_times)

real_back_params = [p[:n_components] for p in back_params[:6]]

# New odor concentrations
new_test_concs = np.asarray([0.5, 1.0])
avg_whiff_conc = np.mean(truncexp1_average(*real_back_params[4:6]))
print("Average whiff concentration: {:.4f}".format(avg_whiff_conc))
new_test_concs *= avg_whiff_conc
n_new_concs = len(new_test_concs)

# Background samples, indexed [time, sample, n_orn]
n_back_samples = 10
conc_samples = sample_ss_conc_powerlaw(
                    *real_back_params, size=n_test_times*(n_back_samples-1), rgen=rgen_meta
                )  # Shaped [sample, component]
back_samples = conc_samples.dot(back_components)
back_samples = back_samples.reshape([n_test_times, n_back_samples-1, -1])
back_samples = np.concatenate([bkvecser_ibcm[test_idx, None, :], back_samples], axis=1)

# TODO: add random ORN activity? for each sample, pull one conc. and one orn choice
extra_back_params = [p[n_components:] for p in back_params[:6]]
extra_concs = sample_ss_conc_powerlaw(
                    *extra_back_params, size=n_test_times*(n_back_samples-1), rgen=rgen_meta
                ).flatten()
orn_choices = rgen_meta.choice(n_dimensions, size=n_test_times*(n_back_samples-1), replace=True)
add_vecs = np.zeros([extra_concs.shape[0], n_dimensions])
add_vecs[np.arange(orn_choices.size), orn_choices] = extra_concs
add_vecs = add_vecs.reshape(n_test_times, n_back_samples-1, n_dimensions)
back_samples[:, 1:] += add_vecs

# Containers for s vectors of each model
mixture_svecs = {a: np.zeros([n_new, n_test_times,  n_new_concs,  
                    n_back_samples, n_dimensions]) for a in sser_dict.keys()}
mixture_tags = {a: SparseNDArray((n_new, n_test_times, n_new_concs,
                    n_back_samples, n_kc), dtype=bool) for a in sser_dict.keys()}
new_odor_tags = sparse.lil_array((n_new, n_kc), dtype=bool)
jaccard_scores = {a: np.zeros([n_new, n_test_times, n_new_concs,  n_back_samples]) 
                  for a in sser_dict.keys()}

In [ ]:
# Compute ideal reduction factor for each concentration
dummy_rgen = np.random.default_rng(0x6e3e2886c30163741daaaf7c8b8a00e6)
ideal_factors = [compute_ideal_factor(c, moments_conc[:2], [n_components, n_dimensions], 
                    generate_odorant, (dummy_rgen,)) for c in new_test_concs]
for i in range(n_new):
    # Compute neural tag of the new odor alone, without inhibition
    new_tag = project_neural_tag(
                    new_odors[i], new_odors[i],
                    proj_mat, **projection_arguments
                )
    new_odor_tags[i, list(new_tag)] = True
    # Parallel and orthogonal components
    x_new_par = find_parallel_component(new_odors[i], 
                        back_components, back_projector)
    x_new_ort = new_odors[i] - x_new_par
    # Now, loop over snapshots, mix the new odor with the back samples,
    # compute the PN response at each test concentration,
    # compute tags too, and save results
    for j in range(n_test_times):
        jj = test_idx[j]
        for k in range(n_new_concs):
            mixtures = (back_samples[j]
                + new_test_concs[k] * new_odors[i])
            # odors, mlx, wmat, 
            # Compute for each model
            mixture_svecs["ibcm"][i, j, k] = ibcm_respond_new_odors(
                mixtures, mser_ibcm[jj], wser_ibcm[jj], 
                ibcm_rates, options=ibcm_options
            )
            mixture_svecs["biopca"][i, j, k] = biopca_respond_new_odors(
                mixtures, [mser_pca[jj], lser_pca[jj], xser_pca[jj]], 
                wser_pca[jj], biopca_rates, options=pca_options
            )
            mixture_svecs["avgsub"][i, j, k] = average_sub_respond_new_odors(
                mixtures, wser_avg[jj], options=avg_options
            )
            mixture_svecs["none"][i, j, k] = mixtures
            mixture_svecs["ideal"][i, j, k] = ideal_linear_inhibitor(
                x_new_par, x_new_ort, mixtures, new_test_concs[k], 
                ideal_factors[k], **avg_options
            )
            for l in range(n_back_samples):
                for mod in mixture_svecs.keys():
                    mix_tag = project_neural_tag(
                        mixture_svecs[mod][i, j, k, l], mixtures[l],
                        proj_mat, **projection_arguments
                    )
                    try:
                        mixture_tags[mod][i, j, k, l, list(mix_tag)] = True
                    except ValueError as e:
                        print(mix_tag)
                        print(mixture_svecs[mod][i, j, k, l])
                        print(proj_mat.dot(mixture_svecs[mod][i, j, k, l]))
                        raise e
                    jaccard_scores[mod][i, j, k, l] = jaccard(mix_tag, new_tag)

In [ ]:
ideal_factors

In [ ]:
# Plot model histogram results
# One plot per new odor concentration
fig, axes = plt.subplots(1, n_new_concs, sharex=True)
fig.set_size_inches(9.5, 4)
axes = axes.flatten()
models = ["none", "ideal", "avgsub", "biopca", "ibcm"]
for m in models:  # Plot IBCM last
    all_jacs = jaccard_scores[m]
    for i in range(n_new_concs):
        hist_outline(
            axes[i], all_jacs[:, :, i, :].flatten(),
            bins="doane", density=True, label=model_nice_names.get(m, m),
            color=model_colors.get(m), alpha=1.0
        )
        axes[i].axvline(
            np.median(all_jacs[:, :, i, :]), ls="--",
            color=model_colors.get(m)
        )
# Labeling the graphs, etc.
for i in range(n_new_concs):
    ax = axes[i]
    axes[i].set_title("New conc. = {:.1f}".format(new_test_concs[i]))
    axes[i].set_xlabel("Jaccard similarity (higher is better)")
    axes[i].set_ylabel("Probability density")
axes[1].legend(loc="upper left", bbox_to_anchor=(1.0, 1.0), frameon=False)
fig.tight_layout()
#fig.savefig("figures/noise_struct/model_comparison_jaccard_transient_noise.pdf",
#            transparent=True, bbox_inches="tight")

plt.show()
plt.close()